In [1]:
import lvm_read
import numpy as np
import pandas as pd
from scipy.stats import *
import glob
import urllib

In [2]:
files = glob.glob("C:/Users/intel/Desktop/Mtech_Proj/Digital twin_29 Jan 2021/dataset/*")
files

['C:/Users/intel/Desktop/Mtech_Proj/Digital twin_29 Jan 2021/dataset\\Bolt1_3_Removed.lvm',
 'C:/Users/intel/Desktop/Mtech_Proj/Digital twin_29 Jan 2021/dataset\\Bolt1_Removed.lvm',
 'C:/Users/intel/Desktop/Mtech_Proj/Digital twin_29 Jan 2021/dataset\\BrokenCage.lvm',
 'C:/Users/intel/Desktop/Mtech_Proj/Digital twin_29 Jan 2021/dataset\\Healthy.lvm',
 'C:/Users/intel/Desktop/Mtech_Proj/Digital twin_29 Jan 2021/dataset\\Vib_Pow_Jammed.lvm',
 'C:/Users/intel/Desktop/Mtech_Proj/Digital twin_29 Jan 2021/dataset\\Vib_Pow_OutterRace.lvm']

In [3]:
for file in files:
    print(file)
    lvm = lvm_read.read(file, read_from_pickle=False)    
  
    data = lvm[0]['data']
#     data = pd.read_csv(file, header = None)
    print(data.shape)

C:/Users/intel/Desktop/Mtech_Proj/Digital twin_29 Jan 2021/dataset\Bolt1_3_Removed.lvm
(3778000, 4)
C:/Users/intel/Desktop/Mtech_Proj/Digital twin_29 Jan 2021/dataset\Bolt1_Removed.lvm
(3620000, 4)
C:/Users/intel/Desktop/Mtech_Proj/Digital twin_29 Jan 2021/dataset\BrokenCage.lvm
(1835299,)
C:/Users/intel/Desktop/Mtech_Proj/Digital twin_29 Jan 2021/dataset\Healthy.lvm
(3600000, 4)
C:/Users/intel/Desktop/Mtech_Proj/Digital twin_29 Jan 2021/dataset\Vib_Pow_Jammed.lvm
(2022000, 4)
C:/Users/intel/Desktop/Mtech_Proj/Digital twin_29 Jan 2021/dataset\Vib_Pow_OutterRace.lvm
(2038000, 4)


In [4]:
lvm.keys()

dict_keys(['Decimal_Separator', 'Writer_Version', 'Reader_Version', 'Separator', 'Multi_Headings', 'X_Columns', 'Time_Pref', 'Operator', 'Date', 'Time', 0, 'Segments'])

In [5]:
lvm[0]['data'][:,3]

array([-0.000867, -0.003498, -0.00284 , ..., -0.000209,  0.000448,
        0.004394])

In [6]:
df_Healthy = pd.DataFrame(lvm[0]['data'])

In [7]:
df_Healthy.shape

(2038000, 4)

In [8]:
# 1757*2048=3598336
df_Healthy[3]

0         -0.000867
1         -0.003498
2         -0.002840
3          0.198068
4          0.005381
             ...   
2037995    0.000119
2037996    0.000777
2037997   -0.000209
2037998    0.000448
2037999    0.004394
Name: 3, Length: 2038000, dtype: float64

In [17]:


number_of_segments = 880*6           
segment_length = 2048

extracted_data_vib = np.empty(shape = (number_of_segments, segment_length))
extracted_data_vib[:] = np.nan

extracted_data_power = np.empty(shape = (number_of_segments, segment_length))
extracted_data_power[:] = np.nan

num = 0
for file in files:
    lvm = lvm_read.read(file, read_from_pickle=False)   
    if  file.endswith('BrokenCage.lvm'):
        lvm_BrokenCage = lvm
        df_BrokenCage = pd.DataFrame(lvm_BrokenCage[0]['data'])
        df_BrokenCage = df_BrokenCage[:-1]
        df_BrokenCage = df_BrokenCage.transform([lambda x:x[0], lambda x:x[1],lambda x:x[2],lambda x:x[3]]).set_axis(['time','vib','time','power'],
                  axis=1,inplace=False)
        data_vib = df_BrokenCage['vib']
        data_power = df_BrokenCage['power']
    else:
        data_vib = lvm[0]['data'][:,3]## Vibration data
        data_power = lvm[0]['data'][:,1]## Power data
    index = 0
    for num_segment in range(880):
        extracted_data_vib[num, :] = data_vib[index*segment_length:(index + 1)*segment_length]
        extracted_data_power[num, :] = data_power[index*segment_length:(index + 1)*segment_length]
        index = index + 1
        num = num + 1

In [12]:
# number_of_segments = 880*6           
# segment_length = 2048

# extracted_data_vib = np.empty(shape = (number_of_segments, segment_length))
# extracted_data_vib[:] = np.nan

# extracted_data_power = np.empty(shape = (number_of_segments, segment_length))
# extracted_data_power[:] = np.nan

# num = 0
# for file in files:
#     lvm = lvm_read.read(file, read_from_pickle=False)    
#     data_vib = lvm[0]['data'][:,3]## Vibration data
#     data_power = lvm[0]['data'][:,1]## Power data
#     index = 0
#     for num_segment in range(987):
#         extracted_data_vib[num, :] = data_vib[index*segment_length:(index + 1)*segment_length]
#         extracted_data_power[num, :] = data_power[index*segment_length:(index + 1)*segment_length]
#         index = index + 1
#         num = num + 1

In [19]:
# for file in files:
#     lvm = lvm_read.read(file, read_from_pickle=False)    
#     data = lvm[0]['data'][:,3]
#     print(data.shape)
#     print(data[0:2048])

In [21]:
# for file in files:
#     lvm = lvm_read.read(file, read_from_pickle=False)    
#     data = lvm[0]['data'][:,1]
#     print(data.shape)
#     print(data[0:2048])

In [22]:
np.sum(np.isnan(extracted_data_vib))
np.sum(np.isnan(extracted_data_power))

0

In [23]:
extracted_data_vib.shape
extracted_data_power.shape

(5280, 2048)

In [24]:
extracted_data_vib = pd.DataFrame(extracted_data_vib)

extracted_data_vib.head()


,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,0.740189,0.868428,1.019027,0.349880,0.562955,0.382762,-0.086463,0.792471,-0.189055,0.119707,...,0.626417,0.178894,-0.144335,0.473845,0.243014,-0.150254,0.333439,-0.146308,-0.406404,0.834560
1,-0.571471,-0.011492,-0.110467,0.214407,0.037831,-0.010835,0.210461,-0.139074,0.127598,0.114446,...,-0.560620,0.281157,0.459048,-0.512941,0.258797,0.039475,0.219668,-0.155844,0.133188,0.180867
2,-0.043716,0.232821,0.015800,0.125954,0.173962,0.008895,0.138778,-0.050293,0.134504,-0.005245,...,0.175277,0.275238,-0.093368,0.390654,0.052299,0.085181,0.205857,0.006922,0.133517,-0.040428
3,0.159823,-0.187082,-0.258435,0.484038,-0.659924,0.230519,0.189416,0.124968,-0.001628,0.089455,...,-0.068378,0.149958,0.006922,0.150616,-0.044374,-0.305457,0.364348,0.140093,-0.106521,0.397888
4,0.303845,0.242356,-0.026618,0.324890,-0.056212,0.294967,0.038160,0.129242,-0.211085,0.523497,...,0.571176,-0.098629,0.110500,0.211447,0.256167,-0.038127,0.357443,-0.030893,0.182840,0.194020


In [25]:
extracted_data_power = pd.DataFrame(extracted_data_power)
extracted_data_power.head()

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,8.918936,5.107902,2.240263,1.841076,1.148254,0.182511,1.161077,0.369609,-0.002614,0.644173,...,0.190732,-0.046676,0.597810,-0.149925,-0.165380,0.341989,0.038817,-0.196617,0.649435,-0.397855
1,0.113788,0.521524,-0.220621,-0.067720,-0.185438,0.277540,-0.022672,0.182840,0.152589,-0.000312,...,0.025007,0.469242,-0.581993,0.529415,0.181525,0.013827,-0.119674,0.085509,0.176592,-0.105863
2,0.345934,-0.099287,0.075316,0.192376,0.165413,0.357443,0.184484,0.406437,0.201583,0.340344,...,0.224271,-0.067391,0.216051,0.133846,0.013827,0.265045,0.200596,0.351853,0.252878,0.269977
3,0.241699,0.315683,0.319629,-0.189055,0.132202,0.430770,-0.235089,0.255180,0.176592,0.412356,...,0.190732,0.369609,0.205857,0.397888,0.132531,0.280170,0.284445,-0.330776,0.735585,0.074658
4,0.240383,0.000016,0.293652,-0.037469,0.267675,0.221969,0.020732,0.049997,0.402491,-0.210099,...,-0.447178,0.610305,0.211447,0.248604,-0.087778,0.308778,-0.002943,0.179881,0.257153,0.019417


In [26]:
extracted_data = pd.concat([extracted_data_power, extracted_data_vib],axis=1)

In [27]:
df = extracted_data.transpose()
df.describe()

,0,1,2,3,4,5,6,7,8,9,...,5270,5271,5272,5273,5274,5275,5276,5277,5278,5279
count,4096.000000,4096.000000,4096.000000,4096.000000,4096.000000,4096.000000,4096.000000,4096.000000,4096.000000,4096.000000,...,4096.000000,4096.000000,4096.000000,4096.000000,4096.000000,4096.000000,4096.000000,4096.000000,4096.000000,4096.000000
mean,0.122462,0.113140,0.109603,0.116170,0.120111,0.114444,0.120136,0.112458,0.116829,0.100300,...,0.598567,0.599232,0.600208,0.592826,0.591739,0.596326,0.603891,0.600652,0.597002,0.591646
std,0.316711,0.263834,0.267383,0.263873,0.270314,0.265292,0.267970,0.269674,0.265180,0.268796,...,0.644538,0.643887,0.646398,0.640251,0.638408,0.638780,0.649187,0.644969,0.647289,0.641911
min,-0.861818,-0.831896,-0.790136,-0.751006,-0.906867,-0.834855,-0.793753,-0.819072,-0.824662,-0.844062,...,-0.785755,-0.661133,-0.632526,-0.672642,-0.711771,-0.724595,-0.688096,-0.583861,-0.647652,-0.778521
25%,-0.041497,-0.038127,-0.039771,-0.037551,-0.034838,-0.033523,-0.031057,-0.042730,-0.037551,-0.043141,...,0.103039,0.103368,0.101066,0.098107,0.098107,0.103039,0.103039,0.103039,0.098765,0.097778
50%,0.125625,0.122173,0.115432,0.118885,0.129242,0.124639,0.130393,0.124639,0.125625,0.110829,...,0.107972,0.108794,0.110931,0.102711,0.103039,0.110109,0.107643,0.109945,0.109287,0.102382
75%,0.273923,0.270059,0.268004,0.272443,0.271374,0.269648,0.272608,0.271292,0.268004,0.260770,...,1.072477,1.082423,1.085054,1.075847,1.085465,1.071902,1.076012,1.091795,1.095741,1.083246
max,8.918936,1.048292,1.011465,1.068350,1.104192,1.046320,1.095971,1.044347,1.116358,1.069666,...,3.171244,3.274164,3.238652,3.285015,3.140663,3.239967,3.168613,3.349135,3.140663,3.273506


In [28]:
features_data = pd.DataFrame()
features_data['x_Mean'] = df.describe().iloc[1]
features_data['x_Std_Deviation'] = df.describe().iloc[2]
features_data['x_Min'] = df.describe().iloc[3]
features_data['x_Max'] = df.describe().iloc[7]
features_data['x_Range'] = features_data['x_Max']-features_data['x_Min']
features_data['x_Sum'] = df.sum()
features_data['x_RMS'] = np.sqrt(np.mean(np.square(df)))
features_data['x_Varience'] = np.var(df)
features_data['x_Kurtosis'] = kurtosis(df)
features_data['x_Crest_Factor'] = features_data['x_Max']/features_data['x_RMS']
features_data['x_Skewness'] = skew(df)
features_data['x_Form_Factor'] = features_data['x_RMS']/features_data['x_Mean']

In [29]:
features_data

,x_Mean,x_Std_Deviation,x_Min,x_Max,x_Range,x_Sum,x_RMS,x_Varience,x_Kurtosis,x_Crest_Factor,x_Skewness,x_Form_Factor
0,0.122462,0.316711,-0.861818,8.918936,9.780754,501.603832,0.339527,0.100281,160.220304,26.268731,6.240298,2.772510
1,0.113140,0.263834,-0.831896,1.048292,1.880188,463.419658,0.287040,0.069591,0.850551,3.652076,-0.081565,2.537044
2,0.109603,0.267383,-0.790136,1.011465,1.801601,448.935140,0.288945,0.071476,0.810392,3.500550,-0.148418,2.636276
3,0.116170,0.263873,-0.751006,1.068350,1.819356,475.831213,0.288283,0.069612,0.874860,3.705903,-0.044382,2.481570
4,0.120111,0.270314,-0.906867,1.104192,2.011059,491.972630,0.295767,0.073052,0.957388,3.733318,-0.015107,2.462457
...,...,...,...,...,...,...,...,...,...,...,...,...
5275,0.596326,0.638780,-0.724595,3.239967,3.964562,2442.550597,0.873811,0.407940,0.090943,3.707860,0.990735,1.465324
5276,0.603891,0.649187,-0.688096,3.168613,3.856709,2473.539047,0.886581,0.421341,0.161423,3.573970,1.020805,1.468113
5277,0.600652,0.644969,-0.583861,3.349135,3.932996,2460.270569,0.881287,0.415883,0.090123,3.800279,0.993584,1.467217
5278,0.597002,0.647289,-0.647652,3.140663,3.788315,2445.320893,0.880507,0.418881,0.032328,3.566880,0.986250,1.474881


In [30]:
fault_Class = []
file_no=0
for file in files:
    file_no = file_no+1
    for num_segment in range(880): #segments for each file
        fault='C'+str(file_no)
        fault_Class.append(fault)


features_data.insert(12,'fault_Class',fault_Class)  

In [31]:
features_data['fault_Class'].value_counts()

C3    880
C4    880
C5    880
C6    880
C2    880
C1    880
Name: fault_Class, dtype: int64

In [32]:
features_data.to_csv('features_digitalTwin_fault.csv')

In [33]:
pwd

'C:\\Users\\intel\\Desktop\\Mtech_Proj\\Digital twin_29 Jan 2021'